In [ ]:
#Data collection code
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os
folder = r"D:\depi project\Data\Y"
os.makedirs(folder, exist_ok=True) 
offset = 20
imgSize = 300
counter = 0
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

while True:
    success, img = cap.read()
    if not success:
        print("❌ Failed to read from camera.")
        break

    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgHeight, imgWidth, _ = img.shape
        x1 = max(x - offset, 0)
        y1 = max(y - offset, 0)
        x2 = min(x + w + offset, imgWidth)
        y2 = min(y + h + offset, imgHeight)

        imgCrop = img[y1:y2, x1:x2]

        if imgCrop.size != 0:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wGap + wCal] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hGap + hCal, :] = imgResize
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)
    cv2.putText(img, f'Images: {counter}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 
                1.5, (255, 0, 255), 3)
    cv2.imshow("Image", img)

    key = cv2.waitKey(1)
    if key == ord("s"):
        if 'imgWhite' in locals():
            counter += 1
            imgName = f'{folder}/Image_{time.time()}.jpg'
            cv2.imwrite(imgName, imgWhite)
            print(f"✅ Saved: {imgName}")
        else:
            print("⚠️ No hand detected. Try again.")
    elif key == ord("q"):
        print("🛑 Quitting...")
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Training model
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
dataset_path = r"D:\depi project\Data"
img_height, img_width = 224, 224
batch_size = 32
epochs = 25
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)
model.save(r"D:\depi project\training model\keras_Model.h5")
class_indices = train_generator.class_indices
with open(r"D:\depi project\training model\labels.txt", "w") as f:
    for label in class_indices:
        f.write(f"{label}\n")


In [ ]:
#test code
import tensorflow.keras as keras
import cv2
import numpy as np
import math
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
from tensorflow.keras.layers import DepthwiseConv2D as KerasDepthwiseConv2D

class CustomDepthwiseConv2D(KerasDepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        groups = kwargs.pop('groups', 1)  
        super().__init__(*args, **kwargs)  
        self.groups = groups  
custom_objects = {
    'DepthwiseConv2D': CustomDepthwiseConv2D
}

class Classifier:
    def __init__(self, labelsPath=None):
        self.labelsPath = labelsPath
        self.model = None  
        if self.labelsPath:
            with open(labelsPath, "r") as f:
                self.labels = [line.strip() for line in f.readlines()]
        else:
            self.labels = []

    def set_model(self, model):
        
        self.model = model

    def getPrediction(self, img, draw=False):
       
        if not self.model:
            raise ValueError("Model is not set. Please set the model using set_model().")
        img_resized = cv2.resize(img, (224, 224)) 
        img_array = np.array(img_resized) / 255.0 
        img_array = np.expand_dims(img_array, axis=0) 

        # Predict using the model
        prediction = self.model.predict(img_array)
        index = np.argmax(prediction, axis=1)[0]

        if draw:
            cv2.putText(img, f"Pred: {self.labels[index]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        return self.labels[index], index

model_path = r"D:\depi project\keras_model.h5"
labels_path = r"D:\depi project\labels.txt"
model = keras.models.load_model(model_path, compile=False, custom_objects=custom_objects)
classifier = Classifier(labelsPath=labels_path)
classifier.set_model(model)  
offset = 20
imgSize = 300
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
while True:
    success, img = cap.read()
    if not success:
        continue

    imgOutput = img.copy()
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        y1 = max(0, y - offset)
        y2 = min(img.shape[0], y + h + offset)
        x1 = max(0, x - offset)
        x2 = min(img.shape[1], x + w + offset)

        imgCrop = img[y1:y2, x1:x2]
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        try:
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wGap + wCal] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hGap + hCal, :] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            label_text = prediction
            cv2.rectangle(imgOutput, (x - offset, y - offset - 50), (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, label_text, (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x - offset, y - offset), (x + w + offset, y + h + offset), (255, 0, 255), 4)

            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)

        except Exception as e:
            print("Resize error:", e)

    cv2.imshow("Image", imgOutput)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
